# World
Implements a world class which integrates an environment with one or more observables and several robots.

Also provides a WorldFactory class that implements a number of standard environments.

In [1]:
import LoggingSetup

from Environment import Environment, PollutionModelEnvironment, EpidemicSpreadEnvironment
from InformationModel import InformationModel, ScalarFieldInformationModel_stored_observation
from Robot import Robot
from Policy import GoToLocationPolicy, FollowPathPolicy, RandomWaypointPolicy

In [2]:
class World:
    def __init__(self, env, im):
        self.env = env
        self.im = im
        self.width, self.height = env.width, env.height
        self.time = 0
        self.robots = []
                
    def add_robot(self, robot):
        self.robots.append(robot)
        
    def enact_policy(self, delta_t = 1.0):
        """For all the agents in the environment, it calls 
        their policies to allow for the scheduling of the actions."""
        for robot in self.robots:
            robot.enact_policy(delta_t)
        
    def proceed(self, delta_t = 1.0):
        """Implements the passing of time"""
        for robot in self.robots:
            robot.proceed(delta_t)
        self.env.proceed(delta_t)
        self.im.proceed(delta_t)
        self.time = self.time + delta_t

# World factory
A utility class for creating worlds for experimentation and papers

In [3]:
class WorldFactory:
    @staticmethod
    def generate_environment_pretrained_static_pollution(width = 10, height = 10, seed = 1):
        """Generates a pollution model that had some polution but now it is static"""
        env = PollutionModelEnvironment("water", width, height, seed)
        env.evolve_speed = 1
        env.p_pollution = 0.1
        for i in range(100):
            env.proceed(1.0)
        # setting it to unchanging
        env.evolve_speed = 0
        env.p_pollution = 0.0
        return env
    
    @staticmethod
    def generate_im(env, settings):
        """Generates an information model appropriate to the environment and the estimation 
        type described by the settings"""
        im = ScalarFieldInformationModel_stored_observation("sample", 
                            env.width, env.height, 
                            estimation_type=settings["estimation_type"])
        return im

    @staticmethod
    def generate_world_pretrained_static_pollution(width = 10, height = 10, 
                                                   seed = 1, estimation_type = "gaussian-process"):
        env = WorldFactory.generate_environment_pretrained_static_pollution(width, height, seed)
        settings = {"estimation_type": estimation_type}
        im = WorldFactory.generate_im(env, settings)
        world = World(env, im)
        return world

In [4]:
world = WorldFactory.generate_world_pretrained_static_pollution()

# A number of experimental work for creating environments, etc. 
Probably obsolete as of August 20, 2021

In [5]:
def generate_robots(settings, world):
    """Generate the robots and their policies according to the settings. 
    Current version generate n robots with random waypoint behavior"""
    count = settings["count"]
    for ib in range(count):
        robot = Robot(f"Robot-{i}", 0, 0, 0, env=world.env, im=world.im)
        robot.policy = RandomWaypointPolicy(None, robot, 1, [0,0], [9, 9])
        world.add_robot(robot)

In [6]:
def run_experiment(genenv, genim, genrob):
    """Generates the environment, the model and the robots"""
    env = genenv()
    im = genim(env)
    world = World(env, im)
    genrob(world)
    record = []
    for t in range(100):
        world.enact_policy(1)
        world.proceed(1)
        ## this is where we get some kind of metric
        record.append(im.score(env))
    return record

In [7]:
# environment model
env = PollutionModelEnvironment("water", 10, 10)
env.evolve_speed = 1
env.p_pollution = 0.1
for i in range(100):
    env.proceed(1.0)
# setting it to unchanging
env.evolve_speed = 0
env.p_pollution = 0.0
im = ScalarFieldInformationModel_stored_observation("sample", env.width, env.height, 
                                estimation_type="gaussian-process")
im.proceed(0)

world = World(env, im)
robi = Robot("Robi", 2, 3, 0, env=env, im=im)
# robot.policy = GoToLocationPolicy(None, robot, 9, 9, 0.2)
world.add_robot(robi)
robur = Robot("Robur", 4, 8, 0, env=env, im=im)
robur.policy = RandomWaypointPolicy(None, robur, 1, [0,0], [9, 9])
world.add_robot(robur)
r2d3 = Robot("R2D3", 5, 9, 0, env=env, im=im)
r2d3.policy = FollowPathPolicy(None, r2d3, 1, [[0,0], [5, 5], [9,0]], repeat = True)
world.add_robot(r2d3)

TypeError: __init__() missing 1 required positional argument: 'seed'

In [ ]:
for i in range(10):
    world.enact_policy(1)
    world.proceed(1)
print(robur.location_history)

In [ ]:
[x[0] for x in robur.location_history]